# Definition

- 주요 국가의 경제를 분석한다.

대상 주요 국가: 미국, 중국, 일본

## 현재 경제 상황

1. 경제 성장
  - GDP 와 Debt
  - 20~30대 취업률
  - 인구 수

2. 유동성, liquidty
  - M2와 M2 Velocity
  
3. 인플레이션
  - 미국의 CPI, 중국의 PPI
  - 글로벌 금 가격
  - 글로벌 부동산 가격

4. 글로벌 교역량
  - 중국의 수출/수입의 비율 및 수출 비율
  - 미국의 주요 무역국의 수입 비중

5. 글로벌 해상 운송량 및 운임료
  - 글로벌 기온
    - 커피 가격, 기온에 큰 영향을 받는 작물
  - 수에즈 / 파나마 운하 운송 량 및 운임



## 패키지

In [8]:
!pip install fredapi
!pip install wbdata
!pip install plotly

In [9]:
from fredapi import Fred
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 판다스 그래프 엔진을 Plotly로 설정
pd.options.plotting.backend = "plotly"

In [10]:
fred = Fred(api_key='1afc3162f75a055edf1d1a95529096cf')

In [11]:
def getDataFred(indicators =[], start:str='1980', end:str='2023')-> pd.DataFrame:
    dictData = {indicator:fred.get_series(indicator, observation_start=start) for indicator in indicators}
    dfData = pd.DataFrame.from_dict(dictData)
    return dfData

## 분석

## GDP

- 한 국가에서 생산된 총 가치
- GDP는 현재 가격에 기반하여 계산되며, 소비자 물가 상승이나 인플레이션의 영향을 받을 수 있다.
- GDP는 경제 활동의 크기를 측정한다.

### Real GDP
- 인플레이션의 영향을 배제하고 경제 성장을 측정
- Real GDP는 특정 기준 연도의 가격 수준을 사용하여 상수 가격에 기반하여 계산
- 실제 생산의 증감에 집중할 수 있다.

### GDP - realGDP
- "GDP - realGDP"의 값은 일종의 인플레이션 수준 또는 물가 상승률을 나타낼 수 있다.
- 일반적으로 "GDP - realGDP"는 GDP에서 실질 GDP를 뺀 값으로, 이는 물가 변동이나 인플레이션의 영향을 반영한다.

In [12]:
indicators = ['GDP', 'GDPC1']
data = getDataFred(indicators)
data.columns= ['GDP', 'realGDP']
data['GDP - realGDP'] = data['GDP'] - data['realGDP']

In [13]:


# 서브플롯 생성
fig = make_subplots(rows=1, cols=2, subplot_titles=("GDP & realGDP", "GDP - realGDP"))

# 왼쪽 서브플롯에 GDP와 realGDP 그래프 추가
fig.add_trace(go.Scatter(x=data.index, y=data['GDP'], name='GDP'), row=1, col=1)
fig.add_trace(go.Scatter(x=data.index, y=data['realGDP'], name='realGDP'), row=1, col=1)

# 오른쪽 서브플롯에 GDP - realGDP 그래프 추가
fig.add_trace(go.Scatter(x=data.index, y=data['GDP - realGDP'], name='GDP - realGDP'), row=1, col=2)

# 레이아웃 설정
fig.update_layout(
    xaxis=dict(title='Year'),
    yaxis=dict(title='($)'),
    title_text="GDP & realGDP vs. GDP - realGDP",
    showlegend=True
    )

fig.show()


- 2012년을 기점으로 GDP-realGDP의 값이 양수로 변했고, 차이가 점점 증가했다.
- 자산 인플레이션이 심화되는 것으로 보인다.

### M2
- M2 통화량은 통화의 확장을 나타내는 지표로, 현금과 유동성이 높은 예금에 시간 예금과 저액 단기 예금 등의 통화성 예금을 추가한 것을 의미
- M2는 경제에서 사용되는 일반적인 통화의 규모를 나타내며, 경제 활동과 통화 공급 사이의 관계를 분석하는 데 사용

### Real M2
- M2 통화량을 인플레이션의 영향을 배제한 실질적인 값으로 나타냄.
- 인플레이션은 물가 상승을 의미하며, Real M2는 가격 수준의 변동을 보정하여 통화량의 실질적인 변화를 측정
- 이를 통해 실질적인 경제 활동과 통화 공급 사이의 관계를 분석하고, 경제 성장과 인플레이션의 영향을 평가하는 데 활용.
- "Real M2 Money Stock"는 인플레이션 조정된 실질 M2 통화량을 나타내는 지표로, 경제의 통화 공급과 관련된 정보를 제공

### M2 Velocity
- M2 통화량의 회전속도를 나타내는 지표
- 보통은 GDP를 M2 통화량으로 나눈 비율
-M2 Velocity는 통화의 유통 속도를 측정하여 경제 활동의 확장 또는 축소를 추적하는 데 사용.

#### M2 Velocity가 감소
- 통화의 회전 속도가 점차 느려진다는 의미
- 통화량이 늘어나더라도 경제 주체들이 통화를 더 느리게 이용하고 소비 및 투자 등 경제 활동이 둔화되는 것을 의미.


In [30]:
indicators = ['M2REAL', 'M2V']
data = getDataFred(indicators)
data.columns= ['Real M2', 'M2 Velocity']
data['M2 Velocity MA 9'] = data['M2 Velocity'].dropna().rolling(window=9).mean()
data['M2 Velocity MA 22'] = data['M2 Velocity'].dropna().rolling(window=22).mean()
data = data.ffill().dropna()

In [65]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 서브플롯 생성
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, subplot_titles=("M2", "Velocity of M2", "Acceleration of M2"))

# 좌측 서브플롯에 M2 그래프 추가
fig.add_trace(go.Bar(x=data.index, y=data['Real M2'], name='M2'), row=1, col=1)
fig.add_trace(go.Scatter(x=data.index, y=data['M2 Velocity'].dropna(), name='M2 Velocity'), row=2, col=1)
fig.add_trace(go.Scatter(x=data.index, y=data['M2 Velocity'].diff().dropna(), name='M2 Acceleration'), row=3, col=1)

# 레이아웃 설정
fig.update_layout(
    xaxis=dict(title='Year'),
    yaxis=dict(title='($)'),
    title_text="M2 & M2 Velocity",
    showlegend=True,
    height=800,
    width=2000,
)

fig.update_layout(
    {
        'yaxis': {'domain': [0.7, 1]},
        'yaxis2': {'domain': [0.35, 0.65]},
        'yaxis3': {'domain': [0, 0.3]},
        'annotations': [
            {'text': 'M2', 'x': 0.5, 'y': 0.98, 'showarrow': False, 'font': {'size': 16}},
            {'text': 'Velocity of M2', 'x': 0.5, 'y': 0.62, 'showarrow': False, 'font': {'size': 16}},
            {'text': 'Acceleration of M2', 'x': 0.5, 'y': 0.26, 'showarrow': False, 'font': {'size': 16}}
        ]
    }
)

fig.show()



### CPI (Consumer Price Index):
- CPI는 다양한 상품과 서비스를 포함하는 지표로, 주거비, 식료품, 의류, 교통, 의료, 교육 등 소비자가 소비하는 다양한 항목을 포함
- 일반적으로 소비자물가상승률을 측정하여 인플레이션 동향을 파악하는 데 사용
- 주로 경제의 물가 안정성과 가계소득의 구매력 변화를 분석하는 데 초점

### CPI_sticky:
- CPI_sticky는 가격이 상대적으로 변동이 적은 상품과 서비스를 중심으로 항목을 선택
- 일반적으로 식료품, 에너지 등 가격 변동이 큰 항목은 포함되지 않음
- 상대적으로 가격 변동이 적은 상품과 서비스의 가격 동향을 파악하여 가격 안정성에 초점
- 가격 변동이 큰 상품이나 서비스에 영향을 받지 않는 경향을 분석하기 위한 지표

### PCE_core (Personal Consumption Expenditures Core Price Index):
- PCE_core는 개인 소비 지출을 측정하고, 식품과 에너지를 제외한 소비 항목을 중심으로 분석
- 주택, 의료, 교육, 교통 등 다양한 소비 항목을 포함
- 주로 인플레이션 동향을 파악하고 경제의 가격 안정성을 분석하는 데 사용
- 식품과 에너지를 제외한 소비 항목의 가격 동향을 중심으로 경제의 물가 상승률을 측정하고 가계소득의 구매력 변화를 이해하는 데 초점

In [79]:
indicators = ['CPIAUCSL', 'CRESTKCPIXSLTRM159SFRBATL', "PCEPI"]
data = getDataFred(indicators)
data.columns= ['CPI', 'CPI_sticky', 'PCE_core']

In [82]:

# 서브플롯 생성
fig = make_subplots(rows=3, cols=1, subplot_titles=("CPI", "CPI_sticky", "PCE_core"))

# 왼쪽 서브플롯에 GDP와 realGDP 그래프 추가
fig.add_trace(go.Scatter(x=data.index, y=data['CPI'], name='CPI'), row=1, col=1)
fig.add_trace(go.Scatter(x=data.index, y=data['CPI_sticky'], name='CPI_sticky'), row=2, col=1)
fig.add_trace(go.Scatter(x=data.index, y=data['PCE_core'], name='PCE_core'), row=3, col=1)

fig.update_layout(
    xaxis=dict(title='Year'),
    title_text=" Inflation",
    showlegend=True,
    height=1000,
    width=2000,
)
fig.show()

In [7]:
SLUEM1524ZS

ValueError: ignored

1. 미국의 경제 성장과 인플레이션의 관계:

- 미국의 경제 성장과 인플레이션 사이에는 일반적으로 양의 상관 관계가 있습니다. 경제 성장이 확대되면 수요가 증가하고 생산량이 늘어나는 경향이 있으며, 이는 물가 상승 압력을 일으킬 수 있습니다.
- 그러나 정확한 관계는 복잡하며, 경제의 구성 요소, 수요와 공급의 변화, 통화 정책 등 다양한 요인이 작용하여 인플레이션에 영향을 미칩니다. 세부적인 관계를 분석하려면 경제 지표와 시계열 분석 방법을 사용하여 연구를 수행해야 합니다.
2. 미국과 일본의 저금리 정책이 중국의 경제 성장에 미치는 영향:

- 미국과 일본의 저금리 정책은 중국의 경제 성장에 영향을 줄 수 있습니다. 저금리 정책은 투자와 대출을 유도하여 경제를 촉진할 수 있으며, 이는 수출과 소비를 증가시킬 수 있습니다.
- 중국은 수출 의존도가 높은 경제로서 미국과 일본의 경기 상황에 민감하게 반응할 수 있습니다. 따라서 미국과 일본의 저금리 정책이 중국의 수출과 경제 성장에 영향을 미칠 수 있습니다.

3. 중국의 생산자 물가와 미국의 인플레이션의 관계:

- 중국의 생산자 물가는 미국의 인플레이션에 영향을 줄 수 있습니다. 중국은 세계적인 제조업 대국으로서 생산자 물가의 변화는 국제 시장에서의 가격 수준에 영향을 미칠 수 있습니다.
- 중국의 물가 상승은 수출 가격 상승으로 이어질 수 있으며, 이는 수입국인 미국에서의 가격 상승 압력을 만들 수 있습니다. 따라서 중국의 생산자 물가 상승은 미국의 인플레이션에 영향을 줄 수 있습니다.

4. 중국의 물가 상승과 미국의 인플레이션의 관계:

- 중국의 물가 상승은 미국의 인플레이션에 영향을 줄 수 있습니다. 중국은 세계에서 가장 큰 제조업 생산 국가로서 중요한 수입국이기도 합니다.
- 중국의 물가 상승은 수출 가격 상승으로 이어질 수 있으며, 이는 미국에서 중국 제품 수입 시 가격 상승 압력을 일으킬 수 있습니다. 따라서 중국의 물가 상승은 미국의 인플레이션에 영향을 줄 수 있습니다.

1. 경제 성장 데이터:

- GDP (국내총생산): 국가별 GDP 데이터를 수집하여 경제 성장 추이를 분석할 수 있습니다. 예를 들어, FRED API나 World Bank API에서 GDP 데이터를 가져올 수 있습니다.
- 경제 지표: 다양한 경제 지표를 활용하여 경제 성장을 분석할 수 있습니다. 예를 들어, 산업 생산지수, 소비자 지출지수, 투자 지출 등을 포함한 데이터를 수집하여 분석에 활용할 수 있습니다.

2. 인플레이션 데이터:

- 소비자 물가지수 (Consumer Price Index, CPI): 소비자 가격 변동을 측정하는 지표로, 국가별 CPI 데이터를 수집하여 인플레이션 추이를 분석할 수 있습니다. FRED API, OECD.Stat API, 또는 국가별 통계청 등에서 CPI 데이터를 가져올 수 있습니다.
- 생산자 물가지수 (Producer Price Index, PPI): 생산자 가격 변동을 측정하는 지표로, 국가별 PPI 데이터를 수집하여 인플레이션과 관련된 분석을 수행할 수 있습니다.

3. 금리 데이터:

- 중앙은행 금리: 각 국가의 중앙은행이 제공하는 기준 금리 데이터를 수집하여 금리 정책과 경제 성장, 인플레이션과의 관계를 분석할 수 있습니다. 예를 들어, FRED API에서 미국 연방준비은행의 기준 금리 데이터를 가져올 수 있습니다.
- 국채 수익률: 국가별 국채 수익률 데이터를 수집하여 시장의 기대와 경제 지표의 변화를 파악하고 금리와 경제 성장, 인플레이션 간의 상관 관계를 분석할 수 있습니다.

GDP 데이터:
World Bank API: World Bank API를 사용하여 국가별 GDP 데이터를 가져올 수 있습니다.
예시 코드:

In [ ]:
import wbdata

# 국가 코드 설정 (예: 미국-USA, 중국-CHN, 일본-JPN)
country_code = "USA"

# GDP 데이터 가져오기
gdp_data = wbdata.get_dataframe(indicator="NY.GDP.MKTP.CD", country=country_code)

# 데이터 출력
print("GDP Data:", gdp_data)


인플레이션 데이터:
FRED API: FRED API를 사용하여 소비자 물가지수(CPI)와 생산자 물가지수(PPI) 데이터를 가져올 수 있습니다.
예시 코드:

In [ ]:
from fredapi import Fred

# FRED API 키 설정
API_KEY = "YOUR_API_KEY"

# Fred 객체 생성
fred = Fred(api_key=API_KEY)

# CPI 데이터 가져오기
cpi = fred.get_series("CPI")

# PPI 데이터 가져오기
ppi = fred.get_series("PPI")

# 데이터 출력
print("CPI Data:", cpi)
print("PPI Data:", ppi)


금리 데이터:
FRED API: FRED API를 사용하여 중앙은행 금리 데이터를 가져올 수 있습니다.
예시 코드:

In [ ]:
from fredapi import Fred

# FRED API 키 설정
API_KEY = "YOUR_API_KEY"

# Fred 객체 생성
fred = Fred(api_key=API_KEY)

# 중앙은행 금리 데이터 가져오기
central_bank_rate = fred.get_series("INTDSRUSM193N")

# 데이터 출력
print("Central Bank Rate Data:", central_bank_rate)


국채 수익률 데이터:
FRED API: FRED API를 사용하여 국가별 국채 수익률 데이터를 가져올 수 있습니다.
예시 코드:

In [ ]:
from fredapi import Fred

# FRED API 키 설정
API_KEY = "YOUR_API_KEY"

# Fred 객체 생성
fred = Fred(api_key=API_KEY)

# CPI 데이터 가져오기
cpi = fred.get_series("CPI")

# PPI 데이터 가져오기
ppi = fred.get_series("PPI")

# 데이터 출력
print("CPI Data:", cpi)
print("PPI Data:", ppi)


#### etx
소비 패턴 분석:

소비자 물가지수(CPI)를 활용하여 미국, 중국, 일본의 소비 패턴을 비교 분석합니다.
소비 지출의 구성을 확인하고, 주요 소비 부문의 변화와 경제 성장률 간의 상관 관계를 알아봅니다.
금융 시장 분석:

글로벌 금 가격과 주요 국가의 경제 성장률 및 인플레이션 지표 간의 상관 관계를 조사합니다.
금융 시장 안정성을 평가하기 위해 금 가격 변동과 주요 국가의 금융 지표를 분석합니다.
국제 무역 패턴 분석:

중국의 주요 무역 파트너국 및 미국의 수입국 비중을 확인하여 중국과 미국의 국제 무역 경향을 비교 분석합니다.
수출 및 수입 비중의 변화와 경제 성장률 사이의 상관 관계를 파악합니다.
기후 변화와 작물 가격 분석:

글로벌 기온 데이터를 활용하여 커피 가격에 영향을 주는 기후 변화를 분석합니다.
기후 변화와 작물 가격의 상관 관계를 조사하여 농작물 시장에 대한 영향을 알아봅니다.
해상 운송 시장 분석:

수에즈 운하와 파나마 운하의 운송량 및 운임료 변동을 분석하여 국제 무역에 대한 영향을 조사합니다.
해상 운송 시장의 동향과 경제 성장률 간의 관계를 확인하여 글로벌 무역 활동을 평가합니다.